<a href="https://colab.research.google.com/github/adityavikramsingh07/Catalog-project/blob/main/hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import files


uploaded = files.upload()

Saving Hate and offensive speech detection.csv to Hate and offensive speech detection (1).csv


In [26]:
# --- Core Data Handling ---
# pandas: For creating and managing our data in a table-like structure called a DataFrame.
import pandas as pd
# numpy: The fundamental package for numerical computation in Python. Deep learning libraries are built on it.
import numpy as np

# --- Text Cleaning Tools ---
# re: Python's 'regular expression' engine. Used for finding and replacing complex text patterns.
import re
# string: A helper module that contains a pre-made string of all punctuation marks.
import string
# nltk.stem.PorterStemmer: A tool from the Natural Language Toolkit for a text normalization technique called stemming.
from nltk.stem import PorterStemmer

# --- Machine Learning & Deep Learning Libraries ---
# sklearn.model_selection.train_test_split: A function to easily split our dataset into training and testing sets.
from sklearn.model_selection import train_test_split
# tensorflow: The core open-source library for deep learning developed by Google.
import tensorflow as tf
# tensorflow.keras...: Keras is a high-level, user-friendly API that runs on top of TensorFlow, making it much easier to build neural networks.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, SpatialDropout1D

# transformers: Hugging Face's state-of-the-art library for models like BERT.
from transformers import BertTokenizer


In [27]:
data = pd.read_csv("Hate and offensive speech detection.csv")


In [29]:
print(data.columns)

Index(['tweet', 'labels'], dtype='object')


In [28]:
# Use 'label' instead of 'class'
data["labels"] = data["label"].map({
    0: "Hate Speech",
    1: "Offensive Language",
    2: "Normal"
})

# Keep only the tweet and the new labels column
data = data[["tweet", "labels"]]

# Check the result
print(data.head())

                                               tweet labels
0                                     Say that Loud!    NaN
1  My heart aches for those who've lost their lov...    NaN
2                                       bitter truth    NaN
3  The people of Gaza deserve a life free from fe...    NaN
4  Oh allah, grant your divine assistance to the ...    NaN


In [32]:
import re
import string
from nltk.stem import PorterStemmer
import pandas as pd # Import pandas here for data loading

stemmer = PorterStemmer()

def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = text.split()
    text = [stemmer.stem(word) for word in text]
    return " ".join(text)

# Re-load the original data to ensure the 'label' column is present for valid_data creation
# This assumes the CSV file is directly accessible by name after upload
data_original = pd.read_csv("Hate and offensive speech detection.csv")

# Ensure 'label' column exists and filter it to create valid_data
# This replicates the necessary part of cell zhwLANYb5x6a for valid_data creation
if 'label' in data_original.columns:
    valid_data = data_original[data_original['label'].isin([0, 1, 2])].copy()
    valid_data["labels"] = valid_data["label"].map({
        0: "Hate Speech",
        1: "Offensive Language",
        2: "Normal"
    })
    # Apply cleaning to the tweet column of valid_data
    valid_data["tweet"] = valid_data["tweet"].apply(clean)
else:
    raise KeyError("The 'label' column was not found in the reloaded CSV data for cleaning.")



In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<UNK>")
tokenizer.fit_on_texts(valid_data["tweet"])

sequences = tokenizer.texts_to_sequences(valid_data["tweet"])

X = pad_sequences(
    sequences,
    maxlen=100,
    padding="post",
    truncating="post"
)


In [36]:
# Ensure 'labels' is categorical with all possible categories, then create one-hot encoded 'y'
# This assumes 'valid_data' is already defined by previous cells (e.g., r7hm3cWoPw-a).
import pandas as pd # Ensure pandas is imported if not already globally available

valid_data['labels'] = pd.Categorical(valid_data['labels'], categories=["Hate Speech", "Offensive Language", "Normal"])
y = pd.get_dummies(valid_data['labels'])

# This cell will now confirm the shape of the processed 'y' DataFrame.
print(y.shape) # This should now be (rows, 3)


(7643, 3)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


In [38]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Convert one-hot y_train back to integers to calculate weights
y_integers = np.argmax(y_train.values, axis=1)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_integers),
    y=y_integers
)
class_weight_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weight_dict)
print("Label distribution in y_train:\n", y_train.sum(axis=0))
print("First 5 rows of y_train:\n", y_train.head())

Class Weights: {0: np.float64(1.7589182968929804), 1: np.float64(0.698583180987203)}
Label distribution in y_train:
 Hate Speech              0
Offensive Language    1738
Normal                4376
dtype: int64
First 5 rows of y_train:
        Hate Speech  Offensive Language  Normal
4298         False               False    True
748          False                True   False
16119        False                True   False
14143        False                True   False
997          False               False    True


In [41]:

# 1. Re-check the original 'label' column from the unmodified `data_original`
# Make sure we only keep rows that have 0, 1, or 2
valid_data = data_original[data_original['label'].isin([0, 1, 2])].copy()

# 2. Re-map
valid_data["labels"] = valid_data["label"].map({
    0: "Hate Speech",
    1: "Offensive Language",
    2: "Normal"
})

# 3. Create One-Hot labels again, but strictly, ensuring all 3 categories are present
valid_data['labels'] = pd.Categorical(valid_data['labels'], categories=["Hate Speech", "Offensive Language", "Normal"])
y = pd.get_dummies(valid_data['labels'])

# 4. Check again - This MUST not have any rows with all zeros
print("New Label Distribution:\n", y.sum())
print("Total Rows:", len(y))

# 5. Re-split with the cleaned data
# X is assumed to be globally available and processed from valid_data['tweet'] as a 0-indexed numpy array.
# Y is a DataFrame that maintains valid_data.index. However, train_test_split will align by row position.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, # Pass X and y directly, they are already aligned by row position.
    test_size=0.2,
    random_state=42
)


New Label Distribution:
 Hate Speech              0
Offensive Language    2139
Normal                5504
dtype: int64
Total Rows: 7643


In [45]:
# 1. Re-check the original 'label' column from `data_original`
# Make sure we only keep rows that have 0, 1, or 2
valid_data = data_original[data_original['label'].isin([0, 1, 2])].copy()

# 2. Re-map
valid_data["labels"] = valid_data["label"].map({
    0: "Hate Speech",
    1: "Offensive Language",
    2: "Normal"
})

# 3. Create One-Hot labels again, but strictly, ensuring all 3 categories are present
valid_data['labels'] = pd.Categorical(valid_data['labels'], categories=["Hate Speech", "Offensive Language", "Normal"])
y = pd.get_dummies(valid_data['labels'])

# 4. Check again - This MUST not have any rows with all zeros
print("New Label Distribution:\n", y.sum())
print("Total Rows:", len(y))

# 5. Re-split with the cleaned data
# X is assumed to be globally available and processed from valid_data['tweet'] as a 0-indexed numpy array.
# Y is a DataFrame that maintains valid_data.index. However, train_test_split will align by row position.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, # Pass X and y directly, they are already aligned by row position.
    test_size=0.2,
    random_state=42
)
from tensorflow.keras.layers import GlobalMaxPooling1D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

vocab_size = len(tokenizer.word_index) + 1

model_final = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100),
    SpatialDropout1D(0.3),

    Conv1D(64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    GlobalMaxPooling1D(),

    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax') # This 3 now matches your 3 dummy columns
])

# Use a standard learning rate now that the data is clean
model_final.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train!
history = model_final.fit(
    X_train, y_train,
    epochs=2,
    batch_size=32,
    validation_data=(X_test, y_test)
)


New Label Distribution:
 Hate Speech              0
Offensive Language    2139
Normal                5504
dtype: int64
Total Rows: 7643
Epoch 1/2
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.6220 - loss: 0.8505 - val_accuracy: 0.7377 - val_loss: 0.8445
Epoch 2/2
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.7273 - loss: 0.6153 - val_accuracy: 0.7384 - val_loss: 0.6121


In [46]:
# Here we're defining the blueprint for our 'Story Teller' expert.
model_lstm = Sequential([
    # Layer 1: The 'Dictionary' Layer.
    Embedding(input_dim=5000, output_dim=128, input_length=100),

    # Layer 2: A 'Forgetfulness' Layer to keep the model from getting too rigid.
    SpatialDropout1D(0.2),

    # Layer 3: The 'Memory' Core of our expert.
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),

    # Layer 4: A 'Thinking' Layer to process what was learned.
    Dense(64, activation='relu'),

    # Layer 5: The final 'Decision' Layer.
    Dense(3, activation='softmax')
])

# Give the expert its learning instructions.
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Begin the training process for our 'Story Teller'.
history_lstm = model_lstm.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


192/192 ━━━━━━━━━━━━━━━━━━━━ 54s 255ms/step - accuracy: 0.6912 - loss: 0.6622 - val_accuracy: 0.7377 - val_loss: 0.5835
Epoch 2/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 49s 256ms/step - accuracy: 0.7090 - loss: 0.6072 - val_accuracy: 0.7377 - val_loss: 0.5791
Epoch 3/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 81s 254ms/step - accuracy: 0.7167 - loss: 0.6032 - val_accuracy: 0.7377 - val_loss: 0.5802
Epoch 4/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 49s 254ms/step - accuracy: 0.7127 - loss: 0.6029 - val_accuracy: 0.7377 - val_loss: 0.5779
Epoch 5/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 83s 259ms/step - accuracy: 0.7142 - loss: 0.6012 - val_accuracy: 0.7377 - val_loss: 0.5771
